## Projet AGIR : modèle de prédiction
#### Rappel de l'objectif
On cherche à prédire si l'évolution en proportion du DPE ou du climat peut influer la consommation électrique, pour permettre à l'Etat d'évaluer la mise en place de politiques sur le DPE ou en anticipation de l'évolution climatique.

#### Prérequis
Afin que le code fonctionne au mieux, faites en sorte de respecter les 3 conditions de mon fichier requirements.txt. Vous pouvez utiliser d'autres versions mais à votre péril ;)

#### Outil proposé
Dans un premier temps, bien relire ce qui a été marqué sur le drive, puis lire le module model.py, qui propose les 3 fonctions : create_model_data, train_model, use_model

#### Etape 1 : jeu de données

Il faudra évidemment le réadapter pour avoir les colonnes souhaitées à l'entraînement. Ici j'ouvre très simplement un fichier à titre d'exemple.

In [10]:
! pip install scikit-learn
! pip install ace_tools

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
import pandas as pd
from model import ModelCreator

ModuleNotFoundError: No module named 'ace_tools'

In [6]:
data = pd.read_csv("data-final-02_04.csv")
data.columns

Index(['N°DPE', 'Etiquette_GES', 'Type_bâtiment', 'Classe_inertie_bâtiment',
       'Adresse', 'Etiquette_DPE', 'N°_région_(BAN)', 'Nom Commune',
       'Code_postal_(brut)', 'Conso_5_usages_é_finale_énergie_n°1',
       'Conso_chauffage_é_finale_énergie_n°1',
       'Type_énergie_principale_chauffage', 'Conso_chauffage_é_primaire',
       'Conso_5_usages_é_primaire', 'Chauffage_électrique', 'Année_x',
       'Code IRIS', 'Nom IRIS', 'Numéro de voie', 'Indice de répétition',
       'Type de voie', 'Libellé de voie', 'Code Commune', 'Segment de client',
       'Nombre de logements',
       'Consommation annuelle totale de l'adresse (MWh)',
       'Consommation annuelle moyenne par logement de l'adresse (MWh)',
       'Consommation annuelle moyenne de la commune (MWh)', 'Code EPCI',
       'Code Région', 'Tri des adresses', 'Température moyenne mensuelle',
       'Température maximale', 'Température minimale', 'Date', 'Saison',
       'Année_y', 'Température moyenne annuelle', 'Centile_2

In [ ]:
# Updated aggregation script according to the new grouping by "N°_région_(BAN)" and keeping the target column

# Copy of the original DataFrame
df_raw = data.copy()

# Target column
target_col = "Consommation annuelle moyenne par logement de l'adresse (MWh)"

# Filter relevant columns
df_filtered = df_raw[[
    "Nom Commune", "Année_y", "Saison", "Etiquette_DPE",
    "Température moyenne mensuelle", "Centile_25%", "Centile_50% (Médiane)",
    "Centile_75%", "N°_région_(BAN)", target_col
]]

# DPE ratio per region-year
dpe_counts = (
    df_filtered
    .groupby(["N°_région_(BAN)", "Année_y", "Etiquette_DPE"])
    .size()
    .unstack(fill_value=0)
)

dpe_columns = ['A', 'B', 'C', 'D', 'E', 'F']
for col in dpe_columns:
    if col not in dpe_counts.columns:
        dpe_counts[col] = 0

dpe_counts["Total"] = dpe_counts[dpe_columns].sum(axis=1)
for col in dpe_columns:
    dpe_counts[f"DPE_{col}"] = dpe_counts[col] / dpe_counts["Total"]

dpe_ratios = dpe_counts[[f"DPE_{col}" for col in dpe_columns]].reset_index()

# Seasonal medians of average temperature
seasonal_medians = (
    df_filtered
    .groupby(["N°_région_(BAN)", "Année_y", "Saison"])["Température moyenne mensuelle"]
    .median()
    .unstack()
    .reset_index()
    .rename(columns={
        "Hiver": "mediane_hiver",
        "Printemps": "mediane_printemps",
        "Été": "mediane_ete",
        "Automne": "mediane_automne"
    })
)

# Annual centiles
annual_centiles = (
    df_filtered
    .groupby(["N°_région_(BAN)", "Année_y"])[["Centile_25%", "Centile_50% (Médiane)", "Centile_75%"]]
    .median()
    .reset_index()
)

# Target value (mean consumption per region-year)
region_targets = (
    df_filtered
    .groupby(["N°_région_(BAN)", "Année_y"])[target_col]
    .mean()
    .reset_index()
)

# Final merge
aggregated_df = dpe_ratios.merge(seasonal_medians, on=["N°_région_(BAN)", "Année_y"], how="left")
aggregated_df = aggregated_df.merge(annual_centiles, on=["N°_région_(BAN)", "Année_y"], how="left")
aggregated_df = aggregated_df.merge(region_targets, on=["N°_région_(BAN)", "Année_y"], how="left")

aggregated_df

,N°DPE,Etiquette_GES,Type_bâtiment,Classe_inertie_bâtiment,Adresse,Etiquette_DPE,N°_région_(BAN),Nom Commune,Code_postal_(brut),Conso_5_usages_é_finale_énergie_n°1,...,DPE_E,DPE_F,mediane_automne,mediane_hiver,mediane_printemps,mediane_ete,Centile_25%_y,Centile_50% (Médiane)_y,Centile_75%_y,Consommation annuelle moyenne par logement de l'adresse (MWh)_y
0,2374E0428703N,D,appartement,Très lourde,7 RUE DES CHARMILLES,D,84.0,annecy,74960,16482.9,...,0.272727,0.092450,12.1,4.5,13.4,21.9,5.625,12.55,18.375,2.283398
1,2374E0428703N,D,appartement,Très lourde,7 RUE DES CHARMILLES,D,84.0,annecy,74960,16482.9,...,0.272727,0.092450,12.1,4.5,13.4,21.9,5.625,12.55,18.375,2.283398
2,2374E0428703N,D,appartement,Très lourde,7 RUE DES CHARMILLES,D,84.0,annecy,74960,16482.9,...,0.272727,0.092450,12.1,4.5,13.4,21.9,5.625,12.55,18.375,2.283398
3,2374E0428703N,D,appartement,Très lourde,7 RUE DES CHARMILLES,D,84.0,annecy,74960,16482.9,...,0.272727,0.092450,12.1,4.5,13.4,21.9,5.625,12.55,18.375,2.283398
4,2374E0428703N,D,appartement,Très lourde,7 RUE DES CHARMILLES,D,84.0,annecy,74960,16482.9,...,0.272727,0.092450,12.1,4.5,13.4,21.9,5.625,12.55,18.375,2.283398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90011,2386E3704884Y,C,appartement,Lourde,16 RUE HENRI DUNANT,C,75.0,poitiers,86000,8390.0,...,0.057143,0.057143,17.0,7.3,12.6,21.3,10.000,14.30,20.375,2.424629
90012,2386E3704884Y,C,appartement,Lourde,16 RUE HENRI DUNANT,C,75.0,poitiers,86000,8390.0,...,0.057143,0.057143,17.0,7.3,12.6,21.3,10.000,14.30,20.375,2.424629
90013,2386E3704884Y,C,appartement,Lourde,16 RUE HENRI DUNANT,C,75.0,poitiers,86000,8390.0,...,0.057143,0.057143,17.0,7.3,12.6,21.3,10.000,14.30,20.375,2.424629
90014,2386E3704884Y,C,appartement,Lourde,16 RUE HENRI DUNANT,C,75.0,poitiers,86000,8390.0,...,0.057143,0.057143,17.0,7.3,12.6,21.3,10.000,14.30,20.375,2.424629


### Préface au modèle : objet ModelCreator

Cet objet est la solution la plus simple pour conserver les données d'une fonction à l'autre. Lisez bien la documentation pour la comprendre.

#### Etape 2 : normalisation des données

La fonction create_model_data va se charger de normaliser les données. Le seul travail à faire de votre côté est la distinction entre les variable quantitatives (scalaires, donc numériques et unidimensionnelles), et les variables catégorielle (autrement dit qualitatives)

In [51]:
mc = ModelCreator(df_main)
mc.create_model_data(column_to_predict="Consommation annuelle moyenne par logement de l'adresse (MWh)", quantitative_variables=["Centile_25%", "Centile_75%"], categorical_variables=["N°_région_(BAN)"])

KeyError: "None of [Index(['Centile_25%', 'Centile_75%'], dtype='object')] are in the [columns]"

#### Etape 3 : Entraînement d'un modèle MLPRegressor

Un MLPRegressor est un réseau de neurones. Un réseau de neurones est une sucession de fonctions polynomiales enveloppées chacune d'une fonction dite "d'activation", la résultante étant une grande fonction. Chaque facteur constant du polynôme est inconnu et est l'objet de l'entraînement. L'entrée de la fonction sont les variables quantitatives et catégorielles choisies, et la sortie de la fonction est la "column_to_predict". Dans notre cas, il s'agit d'un modèle de régression, dans le sens où l'entraînement porte sur une variable quantitative (à l'opposé d'un classifieur où la sortie est une variable catégorielle).

Je propose dans le module de contrôler la "taille" de cette fonction. Chaque élément de la liste "network_layer" ajoute un polynôme, et la valeur indique la taille du polynôme. La valeur max_iterations limite le temps d'entraînement si le modèle à du mal à converger, c'est à dire parvenir à un stade de l'entraînement où le modèle n'évolue plus de façon significative.

Les résultats de la fonction se découpent en 3 parties :
    - La loss curve. C'est une série d'un calcul que l'on appelle la perte pour un modèle. Cette perte mesure le taux d'erreur à l'entraînement, et guide également l'entraînement, car l'ajustement du modèle est fait pour minimiser cette perte. Elle peut être tracée, sachant qu'une courbe de perte idéale est quand elle décroît régulièrement jusqu'à atteindre un plancher (converge) le plus proche de 0 possible
    - Le score d'entraînement. C'est une métrique arbitraire pour mesurer l'efficacité concréte du modèle, basé sur la formule du R². Cette formule pénalise fortement les écarts marqués à la réalité attendue.
    - Le score de test. Pareil que la précédente, mais cette fois appliqué au jeu de données qui a été mis à l'écart de l'entraînement. Il permet de contrôler que le modèle n'a pas juste "appris par coeur" les données d'entraînement.

Nommons le score d'entraînement St celui de test Se :
    - St sera généralement supérieur à Se. On cherche au mieux à faire en sorte que St = Se
    - Se = St = 1 indique que le modèle est parfait. C'est l'utopie qui est visée par nos modèles.
    - Se = 0 indique que le modèle ne fait pas mieux que si on prenait simplement la moyenne des valeurs connues de sortie pour prédire.
    - Se < 0 indique que le modèle est moins performant que la moyenne. Il est alors inenvisageable d'utiliser le modèle.

Je ne sais pas ce que permettra le jeu de données, mais avec le peu de temps qu'on a dépasser Se > 0.50 sera déjà une très bonne preuve de potentiel. Mais ne vous focalisez pas dessus. Se > 0 est la vraie base à viser.

In [47]:
loss_curve, score_train, score_test = mc.train_model(max_iterations=30)
print(min(loss_curve), score_train, score_test)

0.7314258710361997 0.18042290626690116 0.17910929034600287


/Users/xwass/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(


#### Etape 4 : prédiction

Si vous avez un modèle qui marche, vous pourrez proposer une prédiction. L'entrée doit être présentée sous forme d'un dictionnaire. Si vous voulez faire plusieurs prédictions en utilisant toutes les fonctionnalités de ma fonction, demandez moi ou observez le code que j'ai fait.

In [43]:
prediction_input = {
    "Centile_25%": 1,
    "Centile_75%": 10,
    "N°_région_(BAN)": 11
}
mc.use_model(prediction_input)

ValueError: Missing Centile_50% (Médiane)